In [ ]:
import sys
import os
# from google.colab import drive
# drive.mount('/content/drive/')

import time

from math import *
import numpy as np
import matplotlib.pyplot as plt

import cv2
# from google.colab.patches import cv2_imshow
import re

from utils import get_abs_path

In [ ]:
def cv2_imshow(img):
    img = np.concatenate((img[:,:,2:3], img[:,:,1:2], img[:,:,0:1]), axis=2)
    plt.imshow(img)
    plt.show()

In [ ]:
def rand_point_on_map(map_size, offset):

  min_pos = offset
  max_x_pos = map_size[0]-offset
  max_y_pos = map_size[1]-offset

  x_pos = np.random.randint(low = min_pos, high = max_x_pos, size = 1)
  y_pos = np.random.randint(low = min_pos, high = max_y_pos, size = 1)

  return int(x_pos), int(y_pos)

def create_start_point(img, map_size, offset = 3):

  x_pos, y_pos = rand_point_on_map(map_size, offset)

  occupied_pos = 255
  while img[y_pos,x_pos,0] == occupied_pos:
    x_pos, y_pos = rand_point_on_map(map_size, offset)

  cv2.rectangle(img, (x_pos-2, y_pos-2), (x_pos+2, y_pos+2), (0,255,0), -1)
  return x_pos, y_pos

def create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance, offset=3):

  x_pos, y_pos = rand_point_on_map(map_size, offset)

  occupied_pos = 255
  while True:
    x_pos, y_pos = rand_point_on_map(map_size, offset)

    if img[y_pos,x_pos,0] != occupied_pos:
      if sqrt( pow(start_x_pos-x_pos,2) + pow(start_y_pos-y_pos,2) ) > float(min_distance):
        break

  cv2.rectangle(img, (x_pos-2, y_pos-2), (x_pos+2, y_pos+2), (0,255,0), -1)
  return x_pos, y_pos

In [ ]:
def create_rect(img, x_pos, y_pos, width, height):
  x_pos, y_pos, width, height = int(x_pos), int(y_pos), int(width), int(height)
  cv2.rectangle(img, (x_pos, y_pos), (x_pos+width, y_pos+height), (255,0,0), -1)

def add_obstacles(img, rect_count, rect_min, rect_max):
  for i in range(rect_count):
    rect_width = np.random.randint(low = rect_min, high = rect_max, size = 1)
    rect_heigth = np.random.randint(low = rect_min, high = rect_max, size = 1)
    pos = np.random.randint(low = 0, high = map_width-rect_width, size = 4)
    create_rect(img, pos[0], pos[1], rect_width, rect_heigth)

In [ ]:
# image size
map_width = 120
map_height = 120
map_size = [map_width, map_height]

# rect settings
rect_count = 12
rect_min = 15
rect_max = 25

In [ ]:
# generate test image
img = np.zeros((map_width, map_height, 3), np.uint8)

add_obstacles(img, rect_count, rect_min, rect_max)

start_x_pos, start_y_pos = create_start_point(img, map_size)
end_x_pos, end_y_pos = create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance=90)

cv2_imshow(img)

In [ ]:
project_path = get_abs_path(1)
results_path = project_path + '/data/encoder_train/maps/'

In [ ]:
first_map_num = 0
last_map_num = 20000
num_paths = 1
load_maps_from=None # Otherwise set to None

files = []
if not load_maps_from == None:
  for (dirpath, dirnames, filenames) in os.walk(load_maps_from):
    files.extend(filenames)
    break

  files = sorted(files)

  temp_files = []
  visited = []
  for file in files:
    match = re.search(r'map_([\d]*)_', file)
    if not match == None:
      num = int(match.group(1))
      if not num in visited:
        visited.append(num)
        temp_files.append(match.group(0))

  files = temp_files

for n in range(first_map_num, last_map_num):

  image = 0
  # generate image
  if load_maps_from == None:
    image = np.zeros((map_width, map_height, 3), np.uint8)
    # add obstacles
    add_obstacles(image, rect_count, rect_min, rect_max)

  else:
    image = cv2.imread(load_maps_from + files[n] + str(i) + '.png')
    image[:, :, 1] = np.zeros((map_width, map_height), np.uint8)
  
  for i in range(num_paths):
  # create start and end points
    img = np.copy(image)
    start_x_pos, start_y_pos = create_start_point(img, map_size)
    end_x_pos, end_y_pos = create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance=65)

    # create map
    map_name = ''
    if load_maps_from == None:
      map_name = 'map_' + str(n) + '_' + str(i) + '.png'
    else:
      map_name = files[n] + str(i) + '.png'

    map_data = [map_name, str(start_x_pos), str(start_y_pos), str(end_x_pos), str(end_y_pos)]

    print(map_data)
    # cv2_imshow(img)

    # save image
    if not cv2.imwrite(results_path + map_name, img):
      raise Exception('Could not write image')